In [1]:
import torch

In [2]:
import numpy as np

In [3]:
inputs = np.array(
    [[73, 67, 43],
     [91, 88, 64],
     [87, 134, 58],
     [102, 43, 37],
     [69, 96, 70]], dtype='float32'
)

In [4]:
targets = np.array([[56, 70],
                    [81, 101],
                    [119, 133],
                    [22, 37],
                    [103, 119]], dtype='float32')

In [5]:
inputs = torch.from_numpy(inputs)

In [6]:
targets = torch.from_numpy(targets)

In [7]:
inputs.shape

torch.Size([5, 3])

In [8]:
w = torch.randn(2, 3, requires_grad=True)

In [9]:
b = torch.randn(2, requires_grad=True)

In [10]:
def model(x):
    return x @ w.t() + b

In [11]:
preds = model(inputs)

In [14]:
diff = preds - targets
diff_sqr = diff ** 2
torch.sum(diff_sqr) / diff.numel()

tensor(28859.7930, grad_fn=<DivBackward0>)

In [15]:
diff.numel()

10

In [16]:
def mse(t1, t2):
    diff = t1 - t2
    return torch.sum(diff * diff) / diff.numel()

In [115]:
loss = mse(preds, targets)
print(loss)

tensor(667.8197, grad_fn=<DivBackward0>)


In [116]:
# compute gradients
loss.backward()

In [117]:
print(w)
print(w.grad)

tensor([[ 0.8220,  0.6658, -1.0010],
        [-0.1243,  0.7532,  0.5188]], requires_grad=True)
tensor([[  -67.9698, -1135.1125,  -641.9822],
        [ -678.1470,  -906.6447,  -558.3329]])


In [118]:
print(b)
print(b.grad)

tensor([-0.2242,  1.0918], requires_grad=True)
tensor([-4.5138, -8.6968])


In [119]:
# adjust weights & reset gradients
with torch.no_grad():
    w -= w.grad * 1e-5
    b -= b.grad * 1e-5
    w.grad.zero_()
    b.grad.zero_()

In [120]:
preds = model(inputs)
loss = mse(preds, targets)
print(loss)

tensor(637.2733, grad_fn=<DivBackward0>)


In [121]:
import torch.nn as nn

In [122]:
inputs = np.array([[73, 67, 43], [91, 88, 64], [87, 134, 58],
                  [102, 43, 37], [69, 96, 70], [73, 67, 43],
                  [91, 88, 64], [87, 134, 58], [102, 43, 37],
                  [69, 96, 70], [73, 67, 43], [91, 88, 64],
                  [87, 134, 58], [102, 43, 37], [69, 96, 70]],
                 dtype='float32')

In [123]:
targets = np.array([[56, 70], [81, 101], [119, 133],
                   [22, 37], [103, 119], [56, 70],
                   [81, 101], [119, 133], [22, 37],
                   [103, 119], [56, 70], [81, 101],
                   [119, 133], [22, 37], [103, 119]],
                  dtype='float32')

In [124]:
inputs = torch.from_numpy(inputs)

In [126]:
targets = torch.from_numpy(targets)

In [127]:
from torch.utils.data import TensorDataset

In [128]:
train_ds = TensorDataset(inputs, targets)

In [129]:
train_ds[0:3]

(tensor([[ 73.,  67.,  43.],
         [ 91.,  88.,  64.],
         [ 87., 134.,  58.]]),
 tensor([[ 56.,  70.],
         [ 81., 101.],
         [119., 133.]]))

In [130]:
from torch.utils.data import DataLoader

In [131]:
batch_size = 5
train_dl = DataLoader(train_ds, batch_size, shuffle=True)

In [132]:
for xb, yb in train_dl:
    print(xb)
    print(yb)
    break

tensor([[ 73.,  67.,  43.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.],
        [ 73.,  67.,  43.],
        [ 87., 134.,  58.]])
tensor([[ 56.,  70.],
        [ 22.,  37.],
        [103., 119.],
        [ 56.,  70.],
        [119., 133.]])


In [133]:
model = nn.Linear(3, 2)

In [134]:
print(model.weight)

Parameter containing:
tensor([[-0.2078,  0.1972,  0.2424],
        [-0.5698,  0.4959, -0.5525]], requires_grad=True)


In [135]:
print(model.bias)

Parameter containing:
tensor([0.1258, 0.0343], requires_grad=True)


In [136]:
list(model.parameters())

[Parameter containing:
 tensor([[-0.2078,  0.1972,  0.2424],
         [-0.5698,  0.4959, -0.5525]], requires_grad=True),
 Parameter containing:
 tensor([0.1258, 0.0343], requires_grad=True)]

In [137]:
preds = model(inputs)

In [138]:
print(preds)

tensor([[  8.5949, -32.0987],
        [ 14.0865, -43.5451],
        [ 22.5370, -15.1406],
        [ -3.6187, -57.2103],
        [ 21.6897, -30.3564],
        [  8.5949, -32.0987],
        [ 14.0865, -43.5451],
        [ 22.5370, -15.1406],
        [ -3.6187, -57.2103],
        [ 21.6897, -30.3564],
        [  8.5949, -32.0987],
        [ 14.0865, -43.5451],
        [ 22.5370, -15.1406],
        [ -3.6187, -57.2103],
        [ 21.6897, -30.3564]], grad_fn=<AddmmBackward>)


In [139]:
import torch.nn.functional as F

In [140]:
loss_fn = F.mse_loss

In [141]:
loss = loss_fn(model(inputs), targets)

In [142]:
print(loss)

tensor(10774.3467, grad_fn=<MseLossBackward>)


In [143]:
opt = torch.optim.SGD(model.parameters(), lr=1e-5)

In [144]:
def fit(num_epochs, model, loss_fn, opt):
    for epoch in range(num_epochs):
        for xb, yb in train_dl:
            pred = model(xb)
            loss = loss_fn(pred, yb)
            loss.backward()
            opt.step()
            opt.zero_grad()
        if (epoch+1) % 10 == 0:
            print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

In [145]:
fit(100, model, loss_fn, opt)

Epoch [10/100], Loss: 104.3663
Epoch [20/100], Loss: 69.5739
Epoch [30/100], Loss: 64.4654
Epoch [40/100], Loss: 76.7088
Epoch [50/100], Loss: 78.3802
Epoch [60/100], Loss: 75.5884
Epoch [70/100], Loss: 55.2833
Epoch [80/100], Loss: 65.6680
Epoch [90/100], Loss: 43.5019
Epoch [100/100], Loss: 58.0034


In [146]:
preds = model(inputs)

In [147]:
print(preds)

tensor([[ 58.0164,  70.7459],
        [ 81.6431,  94.2531],
        [118.6363, 144.3098],
        [ 25.7686,  40.8544],
        [ 98.2271, 105.9260],
        [ 58.0164,  70.7459],
        [ 81.6431,  94.2531],
        [118.6363, 144.3098],
        [ 25.7686,  40.8544],
        [ 98.2271, 105.9260],
        [ 58.0164,  70.7459],
        [ 81.6431,  94.2531],
        [118.6363, 144.3098],
        [ 25.7686,  40.8544],
        [ 98.2271, 105.9260]], grad_fn=<AddmmBackward>)


In [148]:
print(targets)

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.],
        [ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.],
        [ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])
